In [1]:
import mne
from pyprep.prep_pipeline import PrepPipeline
import numpy as np
import glob

In [7]:
def read_extract(file_path):
    raw=mne.io.read_raw_edf(file_path,preload=True)
    
    montage_channels = ['T4', 'A2', 'C3', 'C4', 'A1', 'T3', 'O1', 'T5', 'O2', 
                    'F8', 'T6', 'FZ', 'FP2', 'F7', 'FP1', 'CZ']
    
    raw.pick_channels(montage_channels)
    
    channel_positions = np.array([
    [5.18e-15, -84.5, -8.85],  # T4
    [3.68e-15, -60.1, -60.1],  # A2
    [3.87e-15, 63.2, 56.9],    # C3
    [-3.87e-15, -63.2, 56.9],  # C4
    [3.68e-15, 60.1, -60.1],   # A1
    [5.18e-15, 84.5, -8.85],   # T3
    [-80.8, 26.1, -4],         # O1
    [-49.9, 68.4, -7.49],      # T5
    [-80.8, -26.1, -4],        # O2
    [49.9, -68.4, -7.49],      # F8
    [-49.9, -68.4, -7.49],     # T6
    [60.7, 0, 59.5],           # FZ
    [80.8, -26.1, -4],         # FP2
    [49.9, 68.4, -7.49],       # F7
    [80.8, 26.1, -4],          # FP1
    [5.2e-15, 0, 85],          # CZ
    [57.6, 48.2, 39.9],        # F3
    [57.6, -48.1, 39.9],       # F4
    [32.9, 0, 78.4]            # FCz
    ])
    available_channels = [ch for ch in montage_channels if ch in raw.info['ch_names']]

    # Adjust positions array to match available channels
    positions_dict = dict(zip(available_channels, channel_positions))
    positions_dict = {ch: positions_dict[ch] for ch in available_channels if ch in positions_dict}
    channel_positions = np.array([positions_dict[ch] for ch in available_channels])

    # Create the montage with the updated channel positions
    montage = mne.channels.make_dig_montage(
        ch_pos=dict(zip(available_channels, channel_positions)), 
        coord_frame='head'
    )
    
    max_time = raw.times[-1]
    tmax1 = min(max_time, 600)
    tmax2 = min(max_time, 1860)
    tmin1 = min(300, max_time)
    tmin2 = min(1500, max_time)
        
    no_stress_interval = raw.copy().crop(tmin=tmin1, tmax=tmax1)
    stressed_interval = raw.copy().crop(tmin=tmin2, tmax=tmax2)
    
    # Initialize the pipeline
    prep_params = {
        'ref_chs': 'eeg',    # Reference channels, or 'average' for average reference
        'reref_chs': 'eeg',      # Channels to rereference
        'line_freqs': [50, 60],  # Frequencies to remove line noise
    }

    # Create the PREP pipeline
    try:
        # Handle the case where there are too many bad channels
        prep = PrepPipeline(no_stress_interval, prep_params, montage)
        prep.fit()

        prep1 = PrepPipeline(stressed_interval, prep_params, montage)
        prep1.fit()

        channel_names = ['FP1', 'FP2', 'F7', 'F8']
        raw1=prep.raw
        raw2=prep1.raw
        
        (raw1).pick_channels(channel_names)
        (raw2).pick_channels(channel_names)
        
        

        epochs_no_stress = mne.make_fixed_length_epochs(raw1, duration=1, overlap=0.5)
        array_no_stressed = epochs_no_stress.get_data()

        epochs_stressed = mne.make_fixed_length_epochs(raw2, duration=1, overlap=0.5)
        array_stressed = epochs_stressed.get_data()

        return array_no_stressed, array_stressed

    except OSError as e:
        print(f"OSError: {str(e)}")
        return None
    except ValueError as e:
        print(f"ValueError: {str(e)}")
        return None

In [8]:
files = glob.glob("EEG files folder/*.edf")

In [9]:
%%capture
no_stressed_data = []
stressed_data = []

for i in files:
    result = read_extract(i)
    if result is not None:
        no_stressed, stressed = result
        no_stressed_data.append(no_stressed)
        stressed_data.append(stressed)
    else:
        print(f"Failed to process file: {i}")


2024-09-23 15:28:56,800 - pyprep.reference - INFO - Bad channels: {'bad_by_nan': [], 'bad_by_flat': [], 'bad_by_deviation': [], 'bad_by_hf_noise': [], 'bad_by_correlation': ['FP2', 'CZ'], 'bad_by_SNR': [], 'bad_by_dropout': [], 'bad_by_ransac': ['T6'], 'bad_all': ['T6', 'CZ', 'FP2']}
2024-09-23 15:28:58,360 - pyprep.reference - INFO - Bad channels: {'bad_by_nan': [], 'bad_by_flat': [], 'bad_by_deviation': [], 'bad_by_hf_noise': [], 'bad_by_correlation': ['T3', 'T4'], 'bad_by_SNR': [], 'bad_by_dropout': [], 'bad_by_ransac': ['F8', 'T6'], 'bad_all': ['T3', 'F8', 'T6', 'T4']}
2024-09-23 15:28:58,388 - pyprep.reference - INFO - Iterations: 1
2024-09-23 15:28:59,936 - pyprep.reference - INFO - Bad channels: {'bad_by_nan': [], 'bad_by_flat': [], 'bad_by_deviation': [], 'bad_by_hf_noise': [], 'bad_by_correlation': ['T3', 'T4'], 'bad_by_SNR': [], 'bad_by_dropout': [], 'bad_by_ransac': ['F8', 'T6'], 'bad_all': ['T3', 'F8', 'T6', 'T4']}
2024-09-23 15:28:59,969 - pyprep.reference - INFO - Iterati

In [10]:
len(stressed_data)

27

In [11]:
len(no_stressed_data)

27

In [12]:
stressed_data[0].shape

(719, 4, 250)

In [13]:
no_stressed_data[1].shape

(599, 4, 250)

In [14]:
channels_names=['FP1','FP2','F7','F8']

In [15]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K

2024-09-23 15:41:31,966 - numexpr.utils - INFO - NumExpr defaulting to 16 threads.


In [16]:
def EEGNet(nb_classes, Chans = 64, Samples = 128, 
             dropoutRate = 0.5, kernLength = 64, F1 = 8, 
             D = 2, F2 = 16, norm_rate = 0.25, dropoutType = 'Dropout'):
    if dropoutType == 'SpatialDropout2D':
        dropoutType = SpatialDropout2D
    elif dropoutType == 'Dropout':
        dropoutType = Dropout
    else:
        raise ValueError('dropoutType must be one of SpatialDropout2D '
                         'or Dropout, passed as a string.')
    
    input1   = Input(shape = (Chans, Samples, 1))

    ##################################################################
    block1       = Conv2D(F1, (1, kernLength), padding = 'same',
                                   input_shape = (Chans, Samples, 1),
                                   use_bias = False)(input1)
    block1       = BatchNormalization()(block1)
    block1       = DepthwiseConv2D((Chans, 1), use_bias = False, 
                                   depth_multiplier = D,
                                   depthwise_constraint = max_norm(1.))(block1)
    block1       = BatchNormalization()(block1)
    block1       = Activation('elu')(block1)
    block1       = AveragePooling2D((1, 4))(block1)
    block1       = dropoutType(dropoutRate)(block1)
    
    block2       = SeparableConv2D(F2, (1, 16),
                                   use_bias = False, padding = 'same')(block1)
    block2       = BatchNormalization()(block2)
    block2       = Activation('elu')(block2)
    block2       = AveragePooling2D((1, 8))(block2)
    block2       = dropoutType(dropoutRate)(block2)
        
    flatten      = Flatten(name = 'flatten')(block2)
    
    dense        = Dense(nb_classes, name = 'dense', 
                         kernel_constraint = max_norm(norm_rate))(flatten)
    softmax      = Activation('softmax', name = 'softmax')(dense)
    
    return Model(inputs=input1, outputs=softmax)




In [17]:
no_stressed = np.vstack(no_stressed_data)
stressed = np.vstack(stressed_data)

In [18]:
no_stressed.shape

(16173, 4, 250)

In [19]:
stressed.shape

(19413, 4, 250)

In [20]:
no_stressed_labels = np.zeros(16173)
stressed_labels = np.ones(19413)


In [21]:
data_list = np.concatenate((no_stressed, stressed), axis=0)

label_list = np.concatenate((no_stressed_labels, stressed_labels), axis=0)

print(data_list.shape)  
print(label_list.shape)

(35586, 4, 250)
(35586,)


In [22]:
no_stressed = no_stressed.reshape((16173, 4, 250, 1))
stressed = stressed.reshape((19413, 4, 250, 1))


data_list = np.concatenate((no_stressed, stressed), axis=0)
label_list = np.concatenate((no_stressed_labels, stressed_labels), axis=0)


In [23]:
# Define the EEGNet model
nb_classes = 2  # Example for binary classification: Stressed vs Not Stressed
model = EEGNet(nb_classes=nb_classes, Chans=4, Samples=250)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


c:\Users\itsay\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [24]:
from tensorflow.keras.utils import to_categorical

# One-hot encode the labels
label_list = to_categorical(label_list, num_classes=nb_classes)


In [25]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data_list, label_list, test_size=0.2, random_state=42)

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/100
890/890 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.6283 - loss: 0.6200 - val_accuracy: 0.5927 - val_loss: 0.7084
Epoch 2/100
890/890 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.6597 - loss: 0.5818 - val_accuracy: 0.6510 - val_loss: 0.5948
Epoch 3/100
890/890 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.6606 - loss: 0.5846 - val_accuracy: 0.6762 - val_loss: 0.6443
Epoch 4/100
890/890 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.6671 - loss: 0.5769 - val_accuracy: 0.6321 - val_loss: 0.6342
Epoch 5/100
890/890 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.6734 - loss: 0.5742 - val_accuracy: 0.6123 - val_loss: 0.7047
Epoch 6/100
890/890 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.6759 - loss: 0.5696 - val_accuracy: 0.6346 - val_loss: 0.6622
Epoch 7/100
890/890 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.6809 - loss: 0.5651 - val_accuracy: 0.6562 - val_loss: 0.6374
Epoch 8/100
890/890 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.6887 - loss: 0.5641 - val_acc

In [26]:
# Evaluate the model on the test set
score = model.evaluate(X_test, y_test)
print(f'Test accuracy: {score[1]}')


223/223 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7969 - loss: 0.3936
Test accuracy: 0.7982579469680786
